<a href="https://colab.research.google.com/github/sipocz/Zenga_DataScience/blob/main/Zenga_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -q playwright nest_asyncio
!playwright install chromium
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

nest_asyncio.apply()


In [12]:
_BASE_URL_='https://www.zenga.hu'
def url_creator(place:str="pest",type:str="haz"):

    return f"{_BASE_URL_}/{place}+elado+{type}?page="

In [13]:
url_creator()

'https://www.zenga.hu/pest+elado+haz?page='

In [22]:
async def scrape_zenga_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page=1):

    start_url = start_url+str(page)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        a_links = await page.query_selector_all("a[data-cy='advert-card-link']") # linkek itt vannak a html-ben
        print(f"Talált linkek száma: {len(a_links)}")

        for a in a_links:
            href = await a.get_attribute("href")
            if href:
                links.append(_BASE_URL_ + "/" + href)

        await browser.close()
        return links

In [24]:
url=url_creator(place='pest-megye',type='haz')
print(url)
zenga_links=[]

# Futtatás
for i in range(5):
    print(f"Page: {i}")
    zenga_page = await scrape_zenga_links(start_url=url,page=i)
    zenga_links.extend(zenga_page)
# Kiírás
for i, link in enumerate(zenga_links, 1):
    print(f"{i}. {link}")


https://www.zenga.hu/pest-megye+elado+haz?page=
Page: 0
Talált linkek száma: 20
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 20
Page: 3
Talált linkek száma: 20
Page: 4
Talált linkek száma: 20
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-gyal/8538660?page=1&pos=1&cr=1000
2. https://www.zenga.hu//ingatlan/elado-csaladi-haz-szentendre/8439624?page=1&pos=2&cr=500
3. https://www.zenga.hu//ingatlan/elado-csaladi-haz-godollo/8440880?page=1&pos=3&cr=500
4. https://www.zenga.hu//ingatlan/elado-csaladi-haz-szentendre/8291448?page=1&pos=4&cr=450
5. https://www.zenga.hu//ingatlan/elado-csaladi-haz-erd/8612360?page=1&pos=5&cr=430
6. https://www.zenga.hu//ingatlan/elado-ikerhaz-csobanka/8398626?page=1&pos=6&cr=425
7. https://www.zenga.hu//ingatlan/elado-csaladi-haz-rackeve/8493427?page=1&pos=7&cr=420
8. https://www.zenga.hu//ingatlan/elado-ikerhaz-diosd/8149373?page=1&pos=8&cr=410
9. https://www.zenga.hu//ingatlan/elado-csaladi-haz-vac-deakvar/8503043?page=1&pos=9&cr=410
10. h

In [31]:
df=pd.DataFrame(zenga_links,columns=['link'])

In [32]:
csv_path='/content/drive/MyDrive/Zenga/'

In [33]:
df.to_csv(csv_path+'links.csv',index=False)

In [5]:
# Futtatás
zenga_links = await scrape_zenga_links("https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",10)

# Kiírás
for i, link in enumerate(zenga_links, 1):
    print(f"{i}. {link}")

Talált linkek száma: 20
1. https://www.zenga.hu/ingatlan/elado-hazresz-neszmely/8623070?page=10&pos=1
2. https://www.zenga.hu/ingatlan/elado-ikerhaz-tatabanya/8550836?page=10&pos=2
3. https://www.zenga.hu/ingatlan/elado-csaladi-haz-esztergom/8418837?page=10&pos=3
4. https://www.zenga.hu/ingatlan/elado-csaladi-haz-komarom-koppanymonostor/8272575?page=10&pos=4
5. https://www.zenga.hu/ingatlan/elado-csaladi-haz-kornye/8565166?page=10&pos=5
6. https://www.zenga.hu/ingatlan/elado-csaladi-haz-aszar/8517621?page=10&pos=6
7. https://www.zenga.hu/ingatlan/elado-csaladi-haz-esztergom/8493673?page=10&pos=7
8. https://www.zenga.hu/ingatlan/elado-sorhaz-bokod/8633353?page=10&pos=8
9. https://www.zenga.hu/ingatlan/elado-sorhaz-tatabanya/8350027?page=10&pos=9
10. https://www.zenga.hu/ingatlan/elado-ikerhaz-aszar/8491089?page=10&pos=10
11. https://www.zenga.hu/ingatlan/elado-csaladi-haz-komarom/8510926?page=10&pos=11
12. https://www.zenga.hu/ingatlan/elado-csaladi-haz-tokod/8536980?page=10&pos=12
13. 

In [44]:
async def scrape_ad_text(url):
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()


        try:
            print(f"🔗 {url}")
            await page.goto(url, timeout=20000)
            try:
                await page.get_by_text("Összes süti engedélyezése").click()
                print("Süti elfogadva")
            except:
                print("Süti gomb nem jelent meg")


            await page.wait_for_timeout(4000)
            # await page.screenshot(path="screenshot.png")  # DEBUG
            desc_element = await page.query_selector("span[data-cy='advert-details-description']")
            desc_text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
            print(f"Szöveg: {desc_text} ")

            price_element = await page.query_selector("div[data-cy='advert-details-price']")
            price_text = await price_element.inner_text() if desc_element else "Nincs ár"
            print(f"Ár: {price_text} ")


            results.append({"url": url, "desc": desc_text, "price":price_text})
        except Exception as e:
            print(f"Hiba: {str(e)}")
            results.append({"url": url, "text": f"Hiba: {str(e)}"})

        await browser.close()

    return results  #return a list


In [48]:
link="https://www.zenga.hu/ingatlan/elado-csaladi-haz-szentendre/8439624"

In [49]:
df = asyncio.run(scrape_ad_text(link))

🔗 https://www.zenga.hu/ingatlan/elado-csaladi-haz-szentendre/8439624
Süti elfogadva
Szöveg: Szentendre Rózsadombján vált elérhetővé egy minimál stílusú luxus villa, melynek bruttó 288 m2 alapterülete van, 837 m2-es panorámás parkosított, medencés telken fekszik egy zsákutcában. Az ingatlan 4 hálószobából, 3 fürdőszobából, konyhából, nappaliból, étkezőből, 3 állásos garázsból, gépészeti helyiségből, tárolóból áll.

Az okos otthon a maga 288 m2- ével 3 szinten osztozik, melynek alsó szintjén.:
-3 állásos garázs,
-gépészeti helyiség,
-tároló,
Földszintjén
-1 szoba,
-nappali,
-gépesített konyha,
-étkező,
-fürdőszoba,
-kamra
-32 m2 terasz, medence kapcsolattal,
Emelet.:
-3 hálószoba,
-1 gardrobe szoba,
-2 fürdőszoba,
-20 m2-es 180 fokos panorámás erkély,

Műszaki tartalma.:
-Vasbeton lemezalap és szerkezeti falak, közte 30 cm XTherm okos tégla,
-Műanyag nyílászárók,
-20 cm hőszigetelés,
-3 fázisú Hűtő-Fűtő hőszivattyú,
-Viessmann 32 KW gázkazán,
-300L HMV Indirekt tároló,
-150 L fűtési-hűté

In [51]:
def scrape_zenga_list(z_list_list):
    out_list=[]
    for url in z_list_list:
        url_data = asyncio.run(scrape_ad_text(url))
        print(url_data)
        out_list.extend(url_data)
    return out_list

In [53]:
zenga_data=scrape_zenga_list(zenga_links)

🔗 https://www.zenga.hu//ingatlan/elado-csaladi-haz-gyal/8538660?page=1&pos=1&cr=1000
Süti elfogadva
Szöveg: Tágas, napfényes családi otthon Gyál csendes, zöldövezeti részén!Eladó egy 120 nm-es, jól karbantartott családi ház Gyálon, kiváló közlekedéssel és infrastruktúrával. Az ingatlan ideális többgyermekes családoknak is: 3 félszoba és 2 teljes méretű szoba, valamint nappali, étkező, konyha, és 2 fürdőszoba biztosítja a kényelmet – az egyik káddal, a másik zuhanyzóval felszerelt. Emellett egy tágas kamra is rendelkezésre áll.Az 583 nm-es, parkosítható telken fedett terasz (16 nm), gyűrűs kút, fedett és nyitott tárolók, valamint fedett kukatároló is található. Az udvaron 2 autó számára van parkolási lehetőség.Fűtés: gázcirkó és vegyes tüzelésű kazán, a nappaliban hűtő-fűtő klíma gondoskodik a kellemes hőmérsékletről nyáron is.A ház 2018-ban felújításon esett át: a vezetékek teljes cseréje, műanyag nyílászárók, új beltéri ajtók kerültek beépítésre. Egy festés után azonnal költözhető!A k

In [55]:
df=pd.DataFrame(zenga_data)
csv_path='/content/drive/MyDrive/Zenga/'
df.to_csv(csv_path+'zenga_data.csv',index=False)

In [58]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [60]:
szoveg=df.desc[0]

In [67]:
def create_prompt(szoveg):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget az alábbi szempontok alapján 1–5-ig,
                            és hozz létre egy összesített eredményt az 1-8 közötti értékek átlagaként ez legyen a 9. Összesítés :
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {

                                "Érthetőség": <szám>,
                                "Részletesség": <szám>,
                                "Szerkezet": <szám>,
                                "Célcsoport": <szám>,
                                "Stílus": <szám>,
                                "Előnyök": <szám>,
                                "Negatívumok": <szám>,
                                "Ösztönzés": <szám>,
                                "Összesítés": <szám.tizedes>"
                            }



                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [68]:
prompt_message=create_prompt(szoveg)


In [63]:
from openai import OpenAI

client = OpenAI(api_key=key)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=prompt_message,
    temperature=0.2,
    max_tokens=200
)

print(response.choices[0].message.content)


{
    "Érthetőség": 5,
    "Részletesség": 5,
    "Szerkezet": 5,
    "Célcsoport": 5,
    "Stílus": 5,
    "Előnyök": 5,
    "Negatívumok": 4,
    "Ösztönzés": 5,
    "Összesítés": 4.88
}


In [64]:
response.choices[0].message.content

'{\n    "Érthetőség": 5,\n    "Részletesség": 5,\n    "Szerkezet": 5,\n    "Célcsoport": 5,\n    "Stílus": 5,\n    "Előnyök": 5,\n    "Negatívumok": 4,\n    "Ösztönzés": 5,\n    "Összesítés": 4.88\n}'

In [65]:
resp_token=response.choices[0]

In [31]:
import json
out_dict= json.loads(response.choices[0].message.content)

In [32]:
out_dict

{'Érthetőség': 4,
 'Részletesség': 5,
 'Szerkezet': 4,
 'Célcsoport': 4,
 'Stílus': 4,
 'Előnyök': 5,
 'Negatívumok': 3,
 'Ösztönzés': 4,
 'Összesítés': 4.25}

In [66]:
df

,url,desc,price
0,https://www.zenga.hu//ingatlan/elado-csaladi-h...,"Tágas, napfényes családi otthon Gyál csendes, ...",84 millió Ft
1,https://www.zenga.hu//ingatlan/elado-csaladi-h...,Szentendre Rózsadombján vált elérhetővé egy mi...,379 millió Ft
2,https://www.zenga.hu//ingatlan/elado-csaladi-h...,Eladó újépítésű családi ház Gödöllőn – 2025 év...,118 millió Ft
3,https://www.zenga.hu//ingatlan/elado-csaladi-h...,"\n \n\nOtthonteremtési támogatás, egyéb hi...","117,9 millió Ft"
4,https://www.zenga.hu//ingatlan/elado-csaladi-h...,"Eladó 3 szintes családi ház Érden,Tusculanumba...",89 millió Ft
...,...,...,...
95,https://www.zenga.hu//ingatlan/elado-ikerhaz-k...,Csak a Dream Credit kínálatában!\n\nKiskunlach...,"52,99 millió Ft"
96,https://www.zenga.hu//ingatlan/elado-csaladi-h...,Eladó családi ház Nagykovácsiban természetköze...,198 millió Ft
97,https://www.zenga.hu//ingatlan/elado-csaladi-h...,"Csak nálunk!\nEladó panorámás, exkluzív család...",145 millió Ft
98,https://www.zenga.hu//ingatlan/elado-csaladi-h...,"Hivatkozási szám: 329858\nSzentendrén, Izbég e...","254,7 millió Ft"
